## 데이타 증강 - 일반대화 포함하여 증강

In [2]:
import os
import pandas as pd
import tensorflow
import numpy as np

In [3]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '../../data', 'raw_csv')
train_data_path = os.path.join(data_dir, 'merged_train_train.csv')

train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,673,직장 내 괴롭힘 대화,김과장 라인 잘 타. 누가 회사 이끄는 건데.\n 아무래도 이건 부당한 것 같습니다...
1,243,협박 대화,이게 어떻게 된거야.? 엄마 수술은 어떻게 된거냐고\n 엄마가 이제 괜찮다고 해서 ...
2,2006,갈취 대화,이사님 이 사진. 모르지 않으실텐데요?\n이. 이건. \n이사님이 여기까지 오신거 ...
3,1073,협박 대화,너 지금 나랑 헤어지자고 한거야? 죽고싶어?\n 그만하자\n 너 내가 헤어지자고 말...
4,893,기타 괴롭힘 대화,철수 씨는 그 학교 나와서 그 연봉 받고 살만해요?\n그게 무슨 말씀이시죠?\n아니...


In [4]:
print('전체 샘플수 :', (len(train_data)))
train_data.info()

전체 샘플수 : 3709
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3709 entries, 0 to 3708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3709 non-null   int64 
 1   class         3709 non-null   object
 2   conversation  3709 non-null   object
dtypes: int64(1), object(2)
memory usage: 87.1+ KB


**한국어 불용어 사전**

In [5]:
# 불용어 제거
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ?.!,]+", " ", sentence)
    
    sentence = sentence.strip()

    return sentence

In [6]:
import re

def preprocess_korean(sentence):
    # 한글 외 모든 문자 제거
    sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', sentence)
    # 연속된 공백 제거
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence


**동의어**

In [7]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)

# 전체': ['전체', '전적', '전부', '총체', '전반']
# '인': ['인', '인간', '인물', '사람', '인류']
#print(wordnet)

In [8]:
# wordnet에서 동의어 리스트를 가져온다. 
# 동의어가 없을 수도 있다
def synonyms_from_wordnet(word):
    synonyms = []
    try:
        for list in wordnet[word] :
             synonyms.append(list)
    except KeyError:
        pass
    
    return synonyms

In [9]:
#'전체': ['전체', '전적', '전부', '총체', '전반']
synonyms = synonyms_from_wordnet( "전체" )
for word in synonyms : 
    print(word)

전체
전적
전부
총체
전반


In [10]:
# word list로 구성된 sentense를 동의어로 바꿔준다
def replace_to_synonyms(input_words, count): 
     
    # new_words
    new_words = []
    
    #
    word_list = list(set([word for word in input_words]))
    random.shuffle(word_list)
    
    replace_count = 0
    for word in word_list : 
        # 동의어 리스트를 가져온다. 
        synonyms = synonyms_from_wordnet( word )
        
        if len(synonyms) == 0 : continue
        
        synonym = random.choice(list(synonyms))
        if( word == synonym) : continue
            
        new_words = [synonym if wd == word else wd for wd in input_words]
        
        replace_count +=1
        
        if replace_count >= count : break
    
    # 
    #if len(new_words) > 0 : 
    #    sentence = ' '.join(new_words)
    #   new_words = sentence.split(" ")
    
       
    return new_words
               

In [11]:
def gen_sentence_by_swap_word(inputs_words, count):
    
    # new_words
    new_words = []
    
    #
#    word_list = list(set([word for word in input_words]))
#    random.shuffle(word_list)
    
    return new_words

In [12]:
def DataAugmentation(sentense, alpha_sr=0.2, aug_count=1):
    
    augmented_sentences = []
    
    # 한글만 처리
    sentense = preprocess_korean(sentense)
    sentense = sentense.split(' ')
    words = [word for word in sentense if word != ""]
    num_words = len(words)
       
    # 동의어로 단어 바꿔 문장 생성하기
    n_sr = max(1, int(alpha_sr*num_words))
    
    for _ in range(aug_count):
        a_words = replace_to_synonyms( words, n_sr )
          
        if( len(a_words) > 0 ):
            augmented_sentences.append(" ".join(a_words))
     
    
    
    # 생성된 문장들 shuffle
    random.shuffle(augmented_sentences)
    return augmented_sentences

In [13]:
new_class_list = []
new_conversation_list = []
new_idx_list = []

idx = len(train_data['conversation']);

filtered_data = train_data

         
for i in range(len(filtered_data['conversation'])):

    class_ = filtered_data['class'][i]
    conv_ = filtered_data['conversation'][i]
    
    num_aug = 3
    
    augmented_sentences = DataAugmentation(conv_, aug_count=num_aug)
    
    if not augmented_sentences : # 리스트가 비어 있으면 True
        continue
        
    for j in range(min(num_aug, len(augmented_sentences))):
        #if not augmented_sentences[j] : continue;
        new_class_list.append( class_ )
        new_conversation_list.append(augmented_sentences[j])
        idx =idx+1
        new_idx_list.append(int(idx))

print(len(new_class_list))    
print(len(new_conversation_list)) 
        

7621
7621


In [14]:
filtered_data['conversation'][0:1]

0    김과장 라인 잘 타. 누가 회사 이끄는 건데.\n 아무래도 이건 부당한 것 같습니다...
Name: conversation, dtype: object

In [15]:
new_conversation_list[0:3]

['김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장이 그렇다하면 어쩔 건데 그러니까 네가 아직도 과장인 거야 저는 하늘 부끄럼 없습니다 요즘 같은 시대에 아직도 이런 사람이 있네 네 알아서 해라 너같은 답답이는 오히려 방해돼 부장님이 그러시면 저는 사내 신고 센터에 갤런 수밖에 없습니다 자네 지금 나 협박하는 거야 자신 있어 두고봐 아니에요 부장님 제가 뭘 하면 되나요 잘 좀 하자 이번 건 안 따오면 아니면 다음 승진도 물 건너 간거다 물론 넌 못할 가능성 높겠지만 네 알겠습니다',
 '김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장이 그렇다하면 어쩔 건데 그러니까 네가 아직도 과장인 거야 저는 하늘 부끄럼 없습니다 요즘 같은 시대에 아직도 이런 사람이 있네 네 알아서 해라 너같은 답답이는 오히려 방해돼 부장님이 그러시면 저는 사내 신고 센터에 갈 수밖에 없습니다 자네 지금 나 협박하는 거야 자신 있어 두고봐 아니에요 부장님 제가 뭘 하면 되나요 잘 좀 하자 이번 건 가운데 따오면 아니면 다음 승진도 물 건너 간거다 물론 넌 못할 가능성 높겠지만 네 알겠습니다',
 '김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장이 그렇다하면 어쩔 건데 그러니까 네가 아직도 과장인 거야 저는 하늘 부끄럼 없습니다 요즘 같은 시대에 아직도 이런 사람이 있네 네 알아서 해라 너같은 답답이는 오히려 방해돼 부장님이 그러시면 저는 수컷 신고 센터에 갈 수밖에 없습니다 자네 지금 나 협박하는 거야 자신 있어 두고봐 아니에요 부장님 제가 뭘 하면 되나요 잘 좀 하자 이번 건 안 따오면 아니면 다음 승진도 물 건너 간거다 물론 넌 못할 가능성 높겠지만 네 알겠습니다']

In [16]:
aug_data = pd.DataFrame([new_idx_list, new_class_list, new_conversation_list],
                  index=['idx','class','conversation'],
                 columns=[x for x in range(len(new_class_list))])
aug_data  = aug_data .transpose()

In [17]:
aug_data.head()

,idx,class,conversation
0,3710,직장 내 괴롭힘 대화,김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장...
1,3711,직장 내 괴롭힘 대화,김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장...
2,3712,직장 내 괴롭힘 대화,김과장 라인 잘 타 누가 회사 이끄는 건데 아무래도 이건 부당한 것 같습니다 김과장...
3,3713,협박 대화,이게 어떻게 된거야 엄마 수술은 어떻게 된거냐고 엄마가 이제 괜찮다고 해서 딱 봐도...
4,3714,협박 대화,이게 어떻게 된거야 엄마 수술은 어떻게 된거냐고 엄마가 이제 괜찮다고 해서 딱 봐도...


In [18]:
aug_train_data = aug_data.sample(frac=1).reset_index(drop=True)

aug_train_data['idx'] = [(len(train_data['conversation'])+x+1)for x in range(len(aug_train_data))]
aug_train_data = aug_train_data.reset_index(drop=True)

In [19]:
aug_train_data.head()

,idx,class,conversation
0,3710,협박 대화,지금 당장 나와 합의하고 해결보자 어떻게 그래 이제껏 내가 당한게 있는데 이렇게 종...
1,3711,기타 괴롭힘 대화,아파 아야 오 표정 썩는 거 봐 아팠어 아 미안 하지마 그만해 이제 니 차례야 니가...
2,3712,일반 대화,이번 여름 방학 계획은 여름이 다가오니까 계획을 세워야 손해 가족과 해외여행 가기로...
3,3713,협박 대화,누가 사람 소문 퍼뜨린거야 무슨소리야 말 안하면 죽여버린다 무슨소문 모른척할래 우리...
4,3714,협박 대화,당신이 먹은 음료엔 독약이 들었어 뭐라구요 무슨 짓입니까 정말인가요 빨리 해킹하지 ...


In [20]:
aug_train_data.tail()

,idx,class,conversation
7616,11326,기타 괴롭힘 대화,쟤 오늘 머리털 좀 봐 왜 완전 떡진 것처럼 딱 달라붙었잖아 머리털 감고 안마른 상...
7617,11327,기타 괴롭힘 대화,돼지야 내 이름은 민경이라구 그러세요 근데 난 왜 돼지로 보이지 이러지 말라구 이거...
7618,11328,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요 저희 회사에서 이 선크림 파는데 한 번 손등에 발...
7619,11329,직장 내 괴롭힘 대화,이대리 오늘 도대체 뭘한거야 창고정리도 하고 바닥도 청소했습니다 이게 한거니 네 열...
7620,11330,기타 괴롭힘 대화,야 대답안하냐 안들리냐 어 연륜 불렀어 어 몇번이나 불렀는데 왜 무시해 무시하는게아...


In [21]:
print(len(aug_train_data))

7621


In [22]:
class_counts = aug_train_data['class'].value_counts()

print(class_counts)

갈취 대화          1833
기타 괴롭힘 대화      1550
협박 대화          1501
직장 내 괴롭힘 대화    1436
일반 대화          1301
Name: class, dtype: int64


In [23]:
aug_train_data.to_csv('../../data/aug_csv/aug_synonym_allclass.csv', index=False, encoding='utf-8-sig')  # index=False는 인덱스를 저장하지 않도록 함

In [24]:
aug_data_path ="../../data/aug_csv/aug_synonym_allclass.csv"
aug_data = pd.read_csv(aug_data_path)
aug_data.head()

,idx,class,conversation
0,3710,협박 대화,지금 당장 나와 합의하고 해결보자 어떻게 그래 이제껏 내가 당한게 있는데 이렇게 종...
1,3711,기타 괴롭힘 대화,아파 아야 오 표정 썩는 거 봐 아팠어 아 미안 하지마 그만해 이제 니 차례야 니가...
2,3712,일반 대화,이번 여름 방학 계획은 여름이 다가오니까 계획을 세워야 손해 가족과 해외여행 가기로...
3,3713,협박 대화,누가 사람 소문 퍼뜨린거야 무슨소리야 말 안하면 죽여버린다 무슨소문 모른척할래 우리...
4,3714,협박 대화,당신이 먹은 음료엔 독약이 들었어 뭐라구요 무슨 짓입니까 정말인가요 빨리 해킹하지 ...


In [25]:
print(aug_data[:1])

    idx  class                                       conversation
0  3710  협박 대화  지금 당장 나와 합의하고 해결보자 어떻게 그래 이제껏 내가 당한게 있는데 이렇게 종...
